In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor, Lambda
from torch import nn
from torchviz import make_dot


BATCH_SIZE = 64
LEARNING_RATE = 0.01
EPOCHS = 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'

##### To train the NN we need:
1. Data
2. Model
3. Loss function
4. Optimizer

In [2]:
# This is a 28 * 28 dataset
training_data = FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    # ToTensor converts the PIL images or NumPy ndarrays into a FloatTensor     
    # and normalizes the image in the range of [0.0, 1.0]
    transform = ToTensor(),
    
    # We need the labels as OHE tensors 
    target_transform = Lambda((lambda x: torch.zeros(10, dtype = torch.float).scatter_(
        dim = 0, index = torch.tensor(x), value = 1)))
)


In [3]:
test_data = FashionMNIST(
    root = 'data',
    train = False,
    download = True,
# ToTensor converts the PIL images or NumPy ndarrays into a FloatTensor 
# and normalizes the image in the range of [0.0, 1.0]
    transform = ToTensor(),
    target_transform = Lambda((lambda x: torch.zeros(10, dtype = torch.float).scatter_(
        dim = 0, index = torch.tensor(x), value = 1)))
)

In [4]:
# Define the 10 classes

lables_map = {
    0: 'T-Shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot',
}

In [5]:
# Pass samples in mini batches

train_dataloader = DataLoader(training_data, batch_size = BATCH_SIZE, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = BATCH_SIZE, shuffle = True)
train_features, train_labels = next(iter(train_dataloader))

In [6]:
# Print the shape of the data

for X, y in train_dataloader:
    print(f'Shape of X: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break

Shape of X: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64, 10]) torch.float32


In [7]:
# Define the model

class NeuralNetwork(nn.Module):
    ''' create a subclass of nn.Modules that inherits some of its functionalities  '''
    def __init__(self):
        ''' Initialise flatten and NN architecture '''
        # Super inherits something from the nn.Module in this case __init__
        super().__init__()
        # Flatten reduces the number of dimensions to 1 (28 * 28 = 784 pixel values)
        self.flatten = nn.Flatten()
        
        # Neural Network architecture
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            # Because of ReLU the nodes do not get activated at the same time
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        # GPU or CPU
        x = x.to(device)
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
# Define the loss function
loss_fn = nn.CrossEntropyLoss()